In [3]:
import pandas as pd
import pvlib
from pvlib.location import Location
from pvlib.pvsystem import PVSystem
from pvlib.modelchain import ModelChain

# Define location parameters
latitude = 35.0  # Replace with your latitude
longitude = -110.0  # Replace with your longitude
tz = 'Etc/GMT+7'  # Replace with your timezone

# Create a location object
location = Location(latitude, longitude, tz=tz)

# Define a specific date and time range
date = '2023-06-21'  # Replace with desired date
times = pd.date_range(start=f'{date} 00:00', end=f'{date} 23:59', freq='1min', tz=tz)

# Get clear sky data for the location
clear_sky = location.get_clearsky(times, model='simplified_solis')

# Define module and inverter parameters (assuming identical for each module)
module_parameters = {
    'pdc0': 300,  # Power at standard test conditions (W)
    'gamma_pdc': -0.004  # Temperature coefficient of power (%/°C)
}
inverter_parameters = {
    'pdc0': 300,  # DC input limit of the inverter (W)
    'pdc': 250  # Max AC output (W)
}

# Define orientations for each module
orientations = [
    {'tilt': 30, 'azimuth': 180},  # South-facing
    {'tilt': 30, 'azimuth': 90},   # East-facing
    {'tilt': 30, 'azimuth': 270},  # West-facing
    {'tilt': 15, 'azimuth': 180},  # South-facing with lower tilt
    # Add more orientations as needed for each of the 10 modules
]

# Initialize a list to collect the power outputs of each module
ac_power_outputs = []

# Calculate power output for each module
for orientation in orientations:
    # Create a PVSystem object with the specific tilt and azimuth
    pv_system = PVSystem(
        module_parameters=module_parameters,
        inverter_parameters=inverter_parameters,
        surface_tilt=orientation['tilt'],
        surface_azimuth=orientation['azimuth']
    )
    
    # Create a ModelChain for the PVSystem and run it
    mc = ModelChain(pv_system, location)
    mc.run_model(clear_sky)
    
    # Append the AC power output of this orientation to the list
    ac_power_outputs.append(mc.ac.fillna(0))  # Fill NaNs with 0 for night hours

# Sum the AC power outputs from each module to get the total system output
total_ac_power_output = sum(ac_power_outputs)

# Print and plot the total AC power output
print(total_ac_power_output)
total_ac_power_output.plot(title='Total PV System Power Output Curve for ' + date, ylabel='AC Power (W)')


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations